In [1]:
from getpass import getpass
import kdbai_client as kdbai
import pandas as pd 
import numpy as np

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [ ]:
#Set up KDB.AI endpoint and API key
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

session = kdbai.Session(endpoint=KDBAI_ENDPOINT, api_key=KDBAI_API_KEY)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [2]:
# session = kdbai.Session(endpoint='http://localhost:8082')

Compatibility with the KDB.AI server is not guaranteed.


In [3]:
db= session.database("default")

In [4]:
n_rows = 100
data = pd.DataFrame({
    'id': np.arange(n_rows, dtype='int16'),
    'content': [f'document{i}' for i in range(n_rows)],
    'publication_date': pd.date_range(start='2020-01-01', periods=n_rows, freq='1MIN'),
    'embeddings': [np.random.rand(25).astype('float32') for _ in range(n_rows)]
})


schema = [
         {'name': 'id', 'type': 'int16'},
         {"name": "content", "type": "str"},
         {'name': 'publication_date', 'type': 'datetime64[ns]'},
         {'name': 'embeddings', 'type': 'float32s'},
         ]


INDEX_FLAT = {'name': 'flat', 'column': 'embeddings', 'type': 'flat', 'params': {'dims': 25, 'metric': 'L2'}}

table = db.create_table("vectorIO",schema=schema, indexes=[INDEX_FLAT])

table.insert(data)

In [6]:
table.query()

,id,content,publication_date,embeddings
0,0,document0,2020-01-01 00:00:00,"[0.30686182, 0.62582034, 0.116534576, 0.334927..."
1,1,document1,2020-01-01 00:01:00,"[0.42317352, 0.7345967, 0.37989786, 0.9099123,..."
2,2,document2,2020-01-01 00:02:00,"[0.7737126, 0.004722189, 0.2140698, 0.6124037,..."
3,3,document3,2020-01-01 00:03:00,"[0.9648716, 0.7883268, 0.80019754, 0.29287186,..."
4,4,document4,2020-01-01 00:04:00,"[0.1775476, 0.44076434, 0.7793666, 0.6005743, ..."
...,...,...,...,...
95,95,document95,2020-01-01 01:35:00,"[0.3033411, 0.1814489, 0.29027653, 0.60969937,..."
96,96,document96,2020-01-01 01:36:00,"[0.44959554, 0.03702071, 0.87624884, 0.9032671..."
97,97,document97,2020-01-01 01:37:00,"[0.7518059, 0.58830845, 0.07086042, 0.59891945..."
98,98,document98,2020-01-01 01:38:00,"[0.7576109, 0.93266773, 0.15977149, 0.05038771..."


## Export Vector DB data using vector-io 

In [ ]:
%run vector-io/src/vdf_io/export_vdf_cli.py kdbai

## Drop table from KDB.AI table

In [17]:
table.drop()

## Try to import the table data into KDB.AI using vector-io

In [ ]:
%run vector-io/src/vdf_io/import_vdf_cli.py  kdbai

## Confirm table exists and data available

In [ ]:
table = db.table("vectorIO")

In [9]:
table.query()

,Embeddings,Sentences
0,"[-0.00248929625377059, 0.006371329538524151, -...","Draft version August 14, 2023\nTypeset using L..."
1,"[0.006372543051838875, 0.002492946805432439, -...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes..."
2,"[0.0013814108679071069, 0.0014908972661942244,...",We define a novel framework: firstly to predic...
3,"[0.014146137051284313, -0.0005565343308262527,...",We predict the spatial and compositional distr...
4,"[0.01576417125761509, 0.004918665625154972, 0....",Selecting ISO water mass\nfraction as an examp...
...,...,...
586,"[-0.00573846697807312, -0.006318436004221439, ...","2021, ApJ, 922, 189,\ndoi: 10.3847/1538-4357/a..."
587,"[0.0037363762967288494, -0.0014329071855172515...","2020,\nNature Methods, 17, 261, doi: 10.1038/s..."
588,"[0.0008434861665591598, -0.01327595580369234, ...","A., Frinchaboy, P. M., et al."
589,"[-0.01480394322425127, 0.0031033621635288, -0....","2013, AJ, 146, 81, doi: 10.1088/0004-6256/146/..."
